### Correlation filter

#### This is the third step of the enrichment calculation

Some Org-IP samples may have very similar profiles, which may lead to underestimation of significance if outlier proteins overlap. 

To tackle this problem, enrichment values are pre-calculated using every other sample in the batch, and these enrichment values are correlated across samples.

The correlation values are then used to filter out highly correlated org-IP samples from being used in the significance test together.

In [1]:
import os
import sys
from datetime import datetime
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

script_path = Path.cwd().parent.parent / "script"
data_path = Path.cwd().parent.parent / "data"
sys.path.append(str(script_path))
from pyseus import primary_analysis as pa
from pyseus import spatial_tools as st

### Load the imputed IP table
The correct datestamp is required to find the input files

In [2]:
%store -r timestamp USE_FROZEN
if USE_FROZEN:
    raise Exception("USE_FROZEN is true, you probably want to skip enrichment and proceed from Fig1")
print(f"Timestamp: {timestamp}")

outprefix = f"{timestamp}_QC_filter_impute"

outdir = Path.cwd() / "output"
preprocessing_out = outdir / f"preprocessing"

Timestamp: 2024-07-14


In [3]:
#### Load bait imputed tables (IPs)
IP_path = preprocessing_out / f"{outprefix}_imputed_table.csv"

try:
    bait_imputed_table = pd.read_csv(IP_path, header=[0, 1], index_col = 0)
except FileNotFoundError:
    print(f"File {IP_path} not found.\nPlease run 1.QC_filter_and_impute.ipynb first or specify the correct timestamp, current value is {timestamp}")
except pd.errors.ParserError:
    print(f"There was an error parsing the CSV file at {IP_path}.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

In [4]:
# #check the WTs
# for i in bait_imputed_table.columns:
#     if "WT" in i[0]:
#         print(i)

### Correlation filter

setup the batches accordin to `2.Batch_selection.ipynb`

In [5]:
batches = [
    ["12", "13"],
    ["01", "03", "04", "05"],
    ["02", "06", "07"],
    ["14", "15", "17"],
    ["09", "10", "11"],
]

In [6]:
# imputed, preprocessed table from the earlier steps
grouped = bait_imputed_table.copy()

# initiate lists to add dataframes for concatenation at the end 
volcano_tables = []
enrichment_tables = []
volcano_tables_againstWT = []
enrichment_tables_againstWT = []

# create a folder to save the correlation tables
correlation_table_dir = outdir / "correlation_tables"
os.makedirs(correlation_table_dir, exist_ok=True)

# iterate through batches
for i, batch in enumerate(batches):
    print('batch: ' + str(i))
    new_cols = []
    # select all the samples from each batch
    for col in list(grouped):
        if col[0] == 'metadata':
            new_cols.append(col)
        # identifying each batch
        elif col[1].split('-')[0] in batch: # splitting the dash and seeing if that experiment is in the batch you're running
            new_cols.append(col) 
    selected = grouped[new_cols].copy()

    #print(list(selected))
    # initial calculation of complement enrichments to identify highly correlated samples
    analysis = pa.AnalysisTables(grouped_table=selected, auto_group=False)
    analysis.generate_export_bait_matrix()

    #analysis.select_wildtype_controls(wt_re='-WT') # select only WT as controls, this is onfirmed by Manu to be the better than using all samples as the intial controls
    analysis.select_matching_WT_control(wt_re='-WT') # select matching WT as controls.

    #print(analysis.exclusion_matrix) # uncomment to check the exclusion matrix
    analysis.exclusion_matrix.to_csv(correlation_table_dir / f"{timestamp}_negative_control_batch_{i}.csv", index=False)

    
    analysis.simple_pval_enrichment(std_enrich=False)
    analysis.convert_to_enrichment_table()
    complements = analysis.enrichment_table.copy()
    control_mat = analysis.exclusion_matrix.copy()

    enrichment_tables_againstWT.append(analysis.enrichment_table.copy()) # export the enrichments against WT
    volcano_tables_againstWT.append(analysis.simple_pval_table.copy()) # export the pvals against WT
    
    # Using the previous calculations, find highly correlated sample (filter is at >0.4)
    spatial = st.SpatialTables(
        preprocessed_table=selected,
        enrichment_table=complements,
        control_mat=control_mat)
    spatial.enrichment_corr_control_mat(corr=0.35)

    # save the correlation table for each batch
    spatial.corr_mat.to_csv(correlation_table_dir / f"{timestamp}_corr_table_batch_{i}.csv", index=False)  
    spatial.sample_corrs.to_csv(correlation_table_dir / f"{timestamp}_corr_val_table_batch_{i}.csv", index=True)
    
    # calculate the final pval and enrichment table for the batch
    spatial.new_corr_ARI(just_enrichment=True, labels=None, reference=None, std_enrich=False)

    # appending respective dataframes to the concatenation list
    volcano_tables.append(spatial.corr_pval_table)
    enrichment_tables.append(spatial.corr_enrichment_table)
    

batch: 0
P-val calculations..
Finished!
P-val calculations..
Finished!
batch: 1
P-val calculations..
Finished!
P-val calculations..
Finished!
batch: 2
P-val calculations..
Finished!
P-val calculations..
Finished!
batch: 3
P-val calculations..
Finished!
P-val calculations..
Finished!
batch: 4
P-val calculations..
Finished!
P-val calculations..
Finished!


The final step is to concatenate results from the batches. 

The enrichment table can be used for multiple purposes, including UMAP generation.

In [7]:
# concatenating the final enrichment table
enrichment_metadata = enrichment_tables[0].loc[:, ["metadata"]].copy()
enrichment_samples = [x.drop(["metadata"], axis=1, level=0) for x in enrichment_tables]

final_enrichments = pd.concat([enrichment_metadata] + enrichment_samples, axis=1)

# concatenating the final volcano table
volcano_metadata = volcano_tables[0].loc[:, ["metadata"]].copy()
volcano_samples = [x.drop(["metadata"], axis=1, level=0) for x in volcano_tables]

final_volcano = pd.concat([volcano_metadata] + volcano_samples, axis=1)

In [8]:
# saving the final tables to files
enrich_out_dir = outdir / "enrichment_and_volcano_tables"
os.makedirs(enrich_out_dir, exist_ok=True)

volcano_csv_path = enrich_out_dir / f"{timestamp}_volcano_table.csv"
enrichment_csv_path = enrich_out_dir / f"{timestamp}_enrichment_table.csv"

final_volcano.to_csv(volcano_csv_path)
final_enrichments.to_csv(enrichment_csv_path)

In [9]:
# check the enrichment table
final_enrichments

metadata  \
                                            Protein IDs   
0         A0A023T6R1;Q96A72;F5H6P7;F5H6N1;F5H3U9;F5H124   
1                          Q9Y5S9;A0A023T787;A0A0J9YW13   
2     A0A0C4DFM1;A0A024QYR3;Q92544;B4DH88;B4DKC1;Q6Z...   
3     A0A024QYR6;A0A1V0DNR7;A0A6G6A825;F6KD02;F6KD01...   
4                              Q99805;A0A024QYR8;B3KSG9   
...                                                 ...   
8594                                             X5D7P8   
8595                                             X5D8X9   
8596  X5DQV1;X5DNI1;B3KV96;E9PD68;B3KXQ5;Q14194;B3KT...   
8597                                             X5DQZ7   
8598                                             X6RHX1   

                                                                          \
                                   Majority protein IDs       Gene names   
0                       A0A023T6R1;Q96A72;F5H6P7;F5H6N1  FLJ10292;MAGOHB   
1                                     Q9Y5S9;A0A023T787       RBM8A;RBM8   
2     A0A0C4DFM1;A0A024QYR3;Q92544;B4DH88;B4DKC1;Q6ZTK5           TM9SF4   
3     A0A024QYR6;A0A1V0DNR7;A0A6G6A825;F6KD02;F6KD01...             PTEN   
4                              Q99805;A0A024QYR8;B3KSG9           TM9SF2   
...                                                 ...              ...   
8594                                             X5D7P8            STK39   
8595                                             X5D8X9          CNTNAP2   
8596  X5DQV1;X5DNI1;B3KV96;E9PD68;B3KXQ5;Q14194;B3KT...            CRMP1   
8597                                             X5DQZ7             GPX1   
8598                                             X6RHX1            SCYL3   

        sample                                                              \
      13-RAB7A   12-RTN4  13-RAB14 12-SEC61B 12-TOMM20 12-PNPLA2   12-ACTB   
0     0.926145 -0.089457  0.277650 -0.486730 -0.736898  0.594597 -0.179236   
1    -1.335100 -2.264500 -0.898950 -0.623300 -2.246100  0.710800  0.860750   
2     5.032826  4.511831  4.840405  4.783531 -0.436809 -4.775289 -4.635126   
3     1.142212  0.064925 -0.376243  0.447371 -0.030686 -0.429486  0.329705   
4     8.977542  6.673675  7.638707  6.847375 -2.183857 -7.775508 -5.963207   
...        ...       ...       ...       ...       ...       ...       ...   
8594  0.660475 -1.063275  0.342780 -0.251286  1.136048  0.383908 -0.365514   
8595  0.064876 -1.042621  0.963853 -0.715858 -0.678593 -0.153141 -0.061779   
8596 -0.581294  0.320812 -0.496372 -0.355636  0.627502  0.130662  0.126567   
8597 -0.664371 -0.571915 -0.755153 -0.186079  1.974409 -0.646730  0.180603   
8598 -0.675243 -0.557370  0.376539  0.393936 -0.696068 -0.282818  0.589653   

      ...                                                                \
      ...   09-PEX3   11-EEA1 09-HSP90AA1 09-ATG101     09-WT 10-TOMM20   
0     ... -0.010676  0.281842   -0.669259  2.692774  2.090668 -0.589472   
1     ... -0.496950 -1.618800   -2.443700 -0.728450  1.836900 -2.969400   
2     ... -0.092300 -1.672800   -5.702337  2.393000  0.315600 -2.685600   
3     ... -1.521976 -0.058161   -0.240339 -0.381822  0.597445  0.105317   
4     ...  0.016550  2.055000   -3.389100 -0.412350 -0.744500 -2.382900   
...   ...       ...       ...         ...       ...       ...       ...   
8594  ... -0.486639  0.510393   -0.336893  1.261714 -0.194651  0.403811   
8595  ...  2.027982  1.111325    0.370447  1.109283 -1.293554 -0.427328   
8596  ...  0.704216  0.332849    0.930923 -0.775370 -0.713403 -0.122160   
8597  ... -0.294442 -0.387396    4.146166 -0.635694  0.105294  3.143259   
8598  ...  0.091349  0.378734   -1.142169  0.164947 -0.721191  0.008961   

                                              
      09-PSMB7  10-EXOC2   09-EDC4  10-VPS35  
0    -1.432714 -0.492929  2.342249 -0.812759  
1    -0.327100  1.017250  1.614550 -0.959200  
2    -4.585382 -4.497900 -0.149000  0.601500  
3    -0.704127 -1.008054 -0.449829 -0.567424  
4    -1.587400  0